In [16]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import pandas as pd
import re

In [30]:
def sanitize_text(text):
    text = re.sub(r'[\[\]\*\?/\\:]', '', text)
    text = ''.join(ch for ch in text if ch.isprintable())
    return text.strip()

def get_blog_full_text(driver, url):
    try:
        driver.get(url)
        time.sleep(2)
        try:
            iframe = driver.find_element(By.TAG_NAME, "iframe")
            driver.switch_to.frame(iframe)
            time.sleep(2)
        except:
            print("⚠️ iframe 없음 또는 전환 실패 → 기본 페이지에서 본문 추출 시도")
        soup = BeautifulSoup(driver.page_source, "html.parser")
        candidates = [
            "div.se-main-container",
            "div#postViewArea",
            "div.post_ct",
            "div.contentArea"
        ]
        for selector in candidates:
            content = soup.select_one(selector)
            if content:
                return sanitize_text(content.get_text(separator="\n"))
            print(f"🌐 현재 URL: {url}")
            print("📦 페이지 구조 HTML 일부:")
            print(driver.page_source[:1000])  # 처음 1000자 출력
        return ""
    except Exception as e:
        print("❌ 본문 추출 오류:", e)
        
        return ""

def crawl_naver_blog(driver, keyword, max_pages=2):
    results = []
    for page in range(1, max_pages + 1):
        start = (page - 1) * 10 + 1
        url = f"https://search.naver.com/search.naver?query={keyword}&where=blog&start={start}"
        driver.get(url)
        time.sleep(2)

        links = driver.find_elements(By.CSS_SELECTOR, "a.api_txt_lines")
        print(f"🔗 Page {page} - 링크 수집 개수: {len(links)}")

        for link in links:
            try:
                title = link.text
                href = link.get_attribute("href")
                if "blog.naver.com" not in href:
                    continue
                print(f"🔗 링크: {href}")
                body = get_blog_full_text(driver, href)
                print(f"📄 {title[:30]} → 본문 길이: {len(body)}")
                print(f"📝 내용 미리보기: {body[:100]}\n")
                # 필터 제거: 모든 결과 저장
                results.append({'keyword': keyword, 'title': title, 'link': href, 'body': body})
            except Exception as e:
                print("⚠️ 링크 처리 오류:", e)
                continue
    return results

def main():
    search_keyword = input("🔍 검색 키워드 입력 (예: 부트캠프): ").strip()

    options = Options()
    # options.add_argument("--headless=new")  # 디버깅 중에는 주석 처리
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--window-size=1920,1080")

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    print(f"▶ '{search_keyword}' 크롤링 시작...")
    results = crawl_naver_blog(driver, search_keyword)

    driver.quit()

    print(f"\n✅ 총 {len(results)}개 결과 수집됨!\n")
    for item in results:
        print(f"[{item['title']}] → {item['link']}")

    df = pd.DataFrame(results)
    df.to_csv("naver_blog_debug_output.csv", index=False, encoding='utf-8-sig')
    print("\n📁 저장 완료: naver_blog_debug_output.csv")

if __name__ == "__main__":
    main()


▶ '부트캠프' 크롤링 시작...
🔗 Page 1 - 링크 수집 개수: 8
🔗 Page 2 - 링크 수집 개수: 8

✅ 총 0개 결과 수집됨!


📁 저장 완료: naver_blog_debug_output.csv


In [8]:
df = pd.read_csv("naver_blog_filtered.csv")

df

,keyword,title,link,body
0,부트캠프 후기,프론트엔드 부트캠프 후기 부탁드립니다. 프론트엔드 부트캠프는 IT 분야에서 취업을 ...,https://kin.naver.com/qna/detail.naver?answerN...,안녕하세요. 답변드립니다.프론트엔드 부트캠프 후기 부탁드립니다.프론트엔드 부트캠프는...
1,부트캠프 후기,데이터 분석 부트 캠프 참여를 고민하고 계시는군요.... 부트 캠프의 1 대 1 밀...,https://kin.naver.com/qna/detail.naver?answerN...,데이터 분석 부트 캠프 참여를 고민하고 계시는군요. 전공자이지만 포트폴리오가 부실해...
2,부트캠프 후기,citeturn0search5 1:1 밀착 관리 및 질문 응답 속도: 두 부트캠프 ...,https://kin.naver.com/qna/detail.naver?answerN...,데이터 분석 포트폴리오를 강화하기 위해 부트캠프 참여를 고려하고 계시군요. 메타코드...
3,부트캠프 후기,"2024.12.10.\n기존에는 디자인 작업 을 주로 했지만, 점점 개발 분야 에 ...",https://nbcamp.spartacodingclub.kr/blog/%EB%94...,NaN
4,부트캠프 후기,"2024.06.21.\n안녕하세요, 코드잇 스프린트입니다. 여러분은 물건 구매를 고...",https://sprint.codeit.kr/blog/%EC%86%94%EC%A7%...,NaN
5,부트캠프 후기,(도봉 SW 2기) 기획부터 배포까지 협업프로젝트로 완성하는 풀스택 웹 개발자 취업...,https://sesac.seoul.kr/course/active/detail.do...,NaN
6,부트캠프 후기,프론트엔드 부트캠프 후기 부탁드립니다. 프론트엔드 부트캠프는 IT 분야에서 취업을 ...,https://kin.naver.com/qna/detail.naver?answerN...,안녕하세요. 답변드립니다.프론트엔드 부트캠프 후기 부탁드립니다.프론트엔드 부트캠프는...
7,부트캠프 후기,데이터 분석 부트 캠프 참여를 고민하고 계시는군요.... 부트 캠프의 1 대 1 밀...,https://kin.naver.com/qna/detail.naver?answerN...,데이터 분석 부트 캠프 참여를 고민하고 계시는군요. 전공자이지만 포트폴리오가 부실해...
8,부트캠프 후기,citeturn0search5 1:1 밀착 관리 및 질문 응답 속도: 두 부트캠프 ...,https://kin.naver.com/qna/detail.naver?answerN...,데이터 분석 포트폴리오를 강화하기 위해 부트캠프 참여를 고려하고 계시군요. 메타코드...
9,부트캠프 후기,"2024.12.10.\n기존에는 디자인 작업 을 주로 했지만, 점점 개발 분야 에 ...",https://nbcamp.spartacodingclub.kr/blog/%EB%94...,NaN
